### Optimizing Temporary Housing Deployment to Reduce U.S. Homelessness 
Nauman Sohani, Yannan Tuo, Charlie Nitschelm

In [3]:
# Load packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Import datasets
# from <data_source> import *
%whos

Variable   Type      Data/Info
------------------------------
GRB        type      <class 'gurobipy._grb.GRB'>
gp         module    <module 'gurobipy' from '<...>\\gurobipy\\__init__.py'>
np         module    <module 'numpy' from 'c:\<...>ges\\numpy\\__init__.py'>
pd         module    <module 'pandas' from 'c:<...>es\\pandas\\__init__.py'>
plt        module    <module 'matplotlib.pyplo<...>\\matplotlib\\pyplot.py'>


In [5]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

# Initialize the model
m = gp.Model(env=env)

In [114]:
# High level variables + assumptions
# Total budget allocated for uplift units
TotalBudget = 14000 # 10000000

# Uplift unit specification assumptions
# Uplift Factory Spec 
FactoryCost = 0

# Uplift THU Spec 
THUCost = 5000

# Uplift Shipping Spec 
THUShippingBase = 100
THUShippingPerMile = 1

# Uplift Operational Spec 
# THU_to_Factory_Max_Distance = 50

In [86]:
# City distance data. What to use as baseline? Generate dictionary for all cities? (500-ish)
city_distance = {"SF": 3000, "Boston": 0}

# use google maps api distance between cities


In [105]:
# CoC Data/homeless population
CoCPopulations = {"SF": 8300, "Boston": 6000}

# Number of cities
NumCities = 2

# city to index dictionary??
# ordered dict for version < 3.7; assumed same order in version >= 3.7
# Todo: update hardcode
def indexToCity(index):
    if index == 0:
        return "SF"
    else:
        return "Boston"

In [120]:
# Decision variables
# How many homes to place in per city
THUs = m.addVars((t for t in range(0, NumCities)), lb=0, name="THUQuantityPerCity")


In [121]:
# Constraints
# Budget
budgetConstr = m.addConstr(sum(THUs[cityIndex]*THUCost + THUs[cityIndex]*city_distance[indexToCity(cityIndex)]*THUShippingPerMile for cityIndex in range(0, NumCities)) <= TotalBudget, name='BudgetConst')

# Unhoused population (allocate no more than # unhoused)
Const4a = m.addConstrs(THUs[cityIndex] <= CoCPopulations[indexToCity(cityIndex)] for cityIndex in range(0, NumCities))


In [122]:
# Objective function
# Maximize number of housing units provided for unhoused populations (ie; minimize unhoused individiuals)
m.setObjective(sum(THUs[city] for city in range(0, NumCities)), gp.GRB.MAXIMIZE)


In [ ]:
# Update and write the model
m.update() # Update model parameters
# m.write("uplift.lp") # Write model to file

# Solve
m.optimize()

# Check model is not infeasible
if m.status == GRB.INFEASIBLE:
    print("model is infeasible")
print("\nObjective value: ", "%.2f" % m.getAttr("ObjVal"))



Objective value:  2.80


In [124]:
# Print solution
for i in THUs:
    # if THUs[i].x > 0:
        print('%s: %g' % (indexToCity(i), THUs[i].x))

SF: 0
Boston: 2.8


#### Analysis of optimization results
beep boop